In [ ]:
#import libraries
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import cv2
from torch.utils.data import DataLoader, Dataset



: 

In [44]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [45]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Mango Leaf Species (Indian)/Compress Data Set'
WIDTH = 224
HEIGHT = 224
n_channels = 3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
x_train = []
x_label = []

dir_list = os.listdir(data_dir)
num_classes = len(dir_list)
print(f"Dataset path: {data_dir}")
print(f"No of Classes: {num_classes}")
for dirname in dir_list:
    dir_path = os.path.join(data_dir, dirname)
    dir_files = os.listdir(dir_path)

    for file_nm in dir_files:
        img_path = os.path.join(dir_path, file_nm)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (WIDTH, HEIGHT))
        x_train.append(img)
        x_label.append(dirname)

# Convert lists to numpy arrays
x_imgs = np.array(x_train)
x_labels = np.array(x_label)
print(f"x_imgs shape: {x_imgs.shape}")
print(f"x_labels shape: {x_labels.shape}")


Dataset path: /content/drive/MyDrive/Mango Leaf Species (Indian)/Compress Data Set
No of Classes: 32
x_imgs shape: (768, 224, 224, 3)
x_labels shape: (768,)


In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_labels = le.fit_transform(x_labels)

split_ratio = 0.95
split_index = int(len(x_imgs) * split_ratio)

# Shuffle the data
indices = np.arange(len(x_imgs))
np.random.shuffle(indices)

x_imgs = x_imgs[indices]
y_labels = y_labels[indices]

# Split the data
x_train = x_imgs[:split_index]
y_train = y_labels[:split_index]

x_valid = x_imgs[split_index:]
y_valid = y_labels[split_index:]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((729, 224, 224, 3), (729,), (39, 224, 224, 3), (39,))

In [55]:
class MangoLeafDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        return image, label

In [56]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resizes the image to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = MangoLeafDataset(x_train, y_train, transform=transform)
valid_dataset = MangoLeafDataset(x_valid, y_valid, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [57]:
class VGG16(nn.Module):
  def __init__(self,num_classes=10):
    super(VGG16,self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.layer2=nn.Sequential(
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.layer3=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU()
    )
    self.layer4=nn.Sequential(
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.layer5=nn.Sequential(
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
    )
    self.layer6=nn.Sequential(
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU()
    )
    self.layer7=nn.Sequential(
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.layer8=nn.Sequential(
        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.layer9=nn.Sequential(
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.layer10=nn.Sequential(
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.layer11=nn.Sequential(
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.layer12=nn.Sequential(
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )
    self.layer13=nn.Sequential(
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.fc=nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7*7*512,4096),
        nn.ReLU()
    )
    self.fc1=nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(4096,4096),
        nn.ReLU()
    )
    self.fc2=nn.Sequential(
        nn.Linear(4096,num_classes)
    )
  def forward(self,x):
      out=self.layer1(x)
      out=self.layer2(out)
      out=self.layer3(out)
      out=self.layer4(out)
      out=self.layer5(out)
      out=self.layer6(out)
      out=self.layer7(out)
      out=self.layer8(out)
      out=self.layer9(out)
      out=self.layer10(out)
      out=self.layer11(out)
      out=self.layer12(out)
      out=self.layer13(out)
      out=out.reshape(out.size(0),-1)
      out=self.fc(out)
      out=self.fc1(out)
      out=self.fc2(out)
      return out

In [58]:
num_classes=32
num_epochs=100
batch_size=16
learning_rate=0.001
model=VGG16(num_classes).to(device)

In [59]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
total_step=len(train_loader)
total_step

23

In [ ]:
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images=images.to(device)
    labels=labels.to(device)
    # print(labels)
    outputs=model(images)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1)%100==0:
      print('Epoch [{}/{}],Step[{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))
torch.save(model.state_dict(), 'model.pth')
with torch.no_grad():
  correct=0
  total=0
  for images,labels in valid_loader:
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)

    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
    del images,labels,outputs
  print('Accuracy of the network on the validation images: {} %'.format(100*correct/total))
